In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)

from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Activation, Embedding, Flatten, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Dot
from keras.layers.advanced_activations import PReLU
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [3]:
print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...


/Users/Alum/Library/Python/3.6/lib/python/site-packages/keras/datasets/imdb.py:49: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


25000 train sequences
25000 test sequences


In [4]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


In [10]:
print('Build model...')
inputs = Input(shape=(maxlen,))
x = inputs
x = Embedding(max_features, 128, dropout=0.2)(x)

x1 = LSTM(32)(x)

x2 = Conv1D(filters=32, kernel_size=3, padding="same")(x)
x2 = GlobalMaxPooling1D()(x2)

x = Dot(axes=-1)([x1, x2])
x = Dense(1)(x)
predictions = Activation("sigmoid")(x)


model = Model(input=inputs, output=predictions)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`
  app.launch_new_instance()


In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 80)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 80, 128)      2560000     input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 80, 32)       12320       embedding_4[0][0]                
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 32)           20608       embedding_4[0][0]                
__________________________________________________________________________________________________
global_max

In [12]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 176s 7ms/step - loss: 0.4164 - acc: 0.8006 - val_loss: 0.3455 - val_acc: 0.8455
Epoch 2/15
25000/25000 [==============================] - 158s 6ms/step - loss: 0.2310 - acc: 0.9086 - val_loss: 0.4037 - val_acc: 0.8402
Epoch 3/15
25000/25000 [==============================] - 162s 6ms/step - loss: 0.1292 - acc: 0.9504 - val_loss: 0.4483 - val_acc: 0.8283
Epoch 4/15
25000/25000 [==============================] - 168s 7ms/step - loss: 0.0703 - acc: 0.9741 - val_loss: 0.6782 - val_acc: 0.8191
Epoch 5/15
25000/25000 [==============================] - 205s 8ms/step - loss: 0.0459 - acc: 0.9841 - val_loss: 0.7660 - val_acc: 0.8242
Epoch 6/15
25000/25000 [==============================] - 196s 8ms/step - loss: 0.0334 - acc: 0.9888 - val_loss: 0.8746 - val_acc: 0.8199
Epoch 7/15
25000/25000 [==============================] - 152s 6ms/step - loss: 0.0230 - acc: 0.9926 - val_loss: 0.9791 - 